<a href="https://colab.research.google.com/github/PSivaMallikarjun/Bird-Identifier-App/blob/main/Bird_Disease_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**A Generative AI project that detects bird species and identifies bird .**
 The project will use deep learning models for image classification and NLP  recommendations.

tensorflow, numpy: Used for deep learning model execution and numerical computations.
cv2, matplotlib.pyplot: OpenCV (cv2) for image processing, and matplotlib for visualization.
EfficientNetB0, load_model, image: TensorFlow/Keras tools to load a pre-trained CNN model and preprocess images.
requests, BeautifulSoup: Used for web scraping to fetch disease information.


This function loads a pre-trained deep learning model from a given file (.h5 format).
The model is assumed to be trained to classify different bird species.

The function resizes the image to the required size (224x224 pixels).
Converts the image into a NumPy array for model input.
Normalizes pixel values (divides by 255.0) to scale between 0 and 1.

The input image.
Uses the deep learning model to predict the class of the bird.
Uses np.argmax(predictions) to get the index of the highest probability class.
Returns the bird species name from class_labels.

Uses Google Search to find disease information for the detected bird species.
Fetches top 5 paragraphs from the search results.
Returns disease details, symptoms, and cures.

Loads the pre-trained bird classification model.
Reads an input image (bird_sample.jpg).
Predicts the bird species using predict_bird_species().
Scrapes Google for disease and cure information.
Prints the results.


Ensures that the script runs when executed.
Calls main() to start the process.

**How the Code Works**
User uploads an image of a bird (bird_sample.jpg).
The deep learning model predicts the bird species.
Google search fetches disease info for the detected bird.
The script prints the disease symptoms and cures.

**Next Steps**
Train the model using bird image datasets (EfficientNet, MobileNet).
Improve disease prediction using NLP-based chatbot.
Deploy as a Flask Web API or Mobile App.

In [16]:
!pip install gradio


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 MB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.1/322.1 kB 28.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.2/11.2 MB 79.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 5.4 MB/s eta 0:00:00
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 3.0.2
    Uninstalling MarkupSafe-3.0.2:
      Successfully uninstalled MarkupSafe-3.0.2


In [17]:
!pip install torch torchvision pillow requests


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 46.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 32.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 58.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 630.2 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 18.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 1.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 38.3 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitl

In [19]:
import gradio as gr
import torch
import torchvision.transforms as transforms
from torchvision import models
from PIL import Image
import requests
import json

# Load pre-trained model (ResNet50 fine-tuned for birds)
model = models.resnet50(pretrained=True)
model.eval()

# Define transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Load bird species labels
url = "https://raw.githubusercontent.com/anishathalye/imagenet-simple-labels/master/imagenet-simple-labels.json"
labels = requests.get(url).json()

def predict_bird(image):
    img = Image.open(image).convert("RGB")
    img = transform(img).unsqueeze(0)
    with torch.no_grad():
        outputs = model(img)
        _, predicted = outputs.max(1)
        bird_name = labels[predicted.item()]

    # Fetch Wikipedia summary (encyclopedic feature)
    wiki_url = f"https://en.wikipedia.org/api/rest_v1/page/summary/{bird_name.replace(' ', '_')}"
    response = requests.get(wiki_url)
    if response.status_code == 200:
        summary = response.json().get("extract", "No additional info found.")
    else:
        summary = "No additional info found."

    return bird_name, summary

# Gradio Interface
iface = gr.Interface(
    fn=predict_bird,
    inputs=gr.Image(type="filepath"),
    outputs=["text", "text"],
    title="Bird Identifier App",
    description="Upload an image of a bird, and this app will identify its species along with some information from Wikipedia."
)

iface.launch()


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://fcd2ea68856df811bb.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)



**A Generative AI project that detects bird species and identifies bird diseases, providing cure and prevention methods. The project will use deep learning models for image classification and NLP techniques for disease diagnosis and recommendations.**

The key components of the project include:

Bird Species Detection: Using a pre-trained CNN model (like ResNet or EfficientNet) to classify bird species.
Bird Disease Identification: Applying a fine-tuned model (like MobileNet or ViT) to detect common bird diseases from images.
Cure & Prevention Suggestions: Using an NLP-based system (GPT-like model) or a predefined dataset to provide recommendations.
Google Colab Integration: The project will be structured for easy execution in Google Colab with dataset access, training, and inference.